# Wilkinson-type polynomials

[Wilkinson-type polynomials](http://en.wikipedia.org/wiki/Wilkinson%27s_polynomial) are of the form

$$W_n(x) = (x-1) \cdot (x-2) \cdot \cdots \cdot (x-n).$$ 

It is well known that it is difficult to calculate the roots of these polynomials. We will investigate how well the `newton` function in the `ValidatedNumerics.jl` package is able to calculate their roots.

We use the `Polynomials` package to calculate the coefficients:

In [1]:
using Polynomials

In [2]:
function wilkinson_coefficients(n)
    p = poly(collect(1:n))  # define a polynomial by its roots
    p.a  # the coefficients
end

wilkinson_coefficients (generic function with 1 method)

For example, $W_2$ is given by $W_2(x) = 2 - 3x + x^2$; note the order of the coefficients:

In [3]:
wilkinson_coefficients(2)

3-element Array{Int64,1}:
  2
 -3
  1

[Horner's method](http://en.wikipedia.org/wiki/Horner%27s_method) is an efficient method for evaluating polynomials. For $W_2$, the idea is to rewrite the polynomial as 

$W_2(x) = (x + 3)\cdot x + 2,$

so that no explicit powers remain.

## Generating the polynomials 

For convenience, we repeat here code from `test/root_finding_tests/wilkinson_polynomials.jl` to generate efficient versions of the Wilkinson polynomials using metaprogramming to implement Horner's rule

In [4]:
function generate_wilkinson_horner(n)
    coeffs = wilkinson_coefficients(n)

    expr = :($(coeffs[end]))  # start with highest power

    for i in length(coeffs)-1:-1:1
        expr = :($expr*x + $(coeffs[i]))
    end

    expr
end

generate_wilkinson_horner (generic function with 1 method)

For example, the code generated for $W_3$ is

In [5]:
generate_wilkinson_horner(3)

:(((1x + -6) * x + 11) * x + -6)

We now generate various Wilkinson polynomials:

In [6]:
function subscriptify(n::Int)
    subscript_digits = [c for c in "₀₁₂₃₄₅₆₇₈₉"]
    dig = reverse(digits(n))
    join([subscript_digits[i+1] for i in dig])
end

for n in 1:15
    fn_name = symbol(string("W", subscriptify(n)))
    expr = generate_wilkinson_horner(n)

    @eval $(fn_name)(x) = $(expr)
end

We can now refer to the polynomials using `W₁`, `W₂`, etc.:

In [7]:
x = [0, 1, 2, 3, 4, 10]
map(W₃, x)

6-element Array{Int64,1}:
  -6
   0
   0
   0
   6
 504

We see that indeed `W₃` has roots at $1$, $2$ and $3$.

## Rigorous root finding

We can now apply the `newton` function from `ValidatedNumerics` to search for roots of the function within a given interval as follows. (In the following, it is necessary to run certain cells twice to eliminate the compilation overhead in the timings.)

In [8]:
using ValidatedNumerics

In [9]:
@time roots = newton(W₃, @interval(-10, 10))

elapsed time: 1.630687196 seconds (54897236 bytes allocated, 4.59% gc time)


3-element Array{(Interval{Float64},Symbol),1}:
 ([0.9999999999999994, 1.0000000000000009],:unique)
 ([1.999999999999999, 2.0000000000000027],:unique) 
 ([2.999999999999997, 3.0000000000000004],:unique) 

We see that `newton` correctly finds the roots; it also *guarantees* that they are unique, i.e. that there is exactly one root in the given interval. We can obtain higher precision by changing the precision in the calculation:

In [10]:
setprecision(Interval, 256)
@time roots2 = newton(W₃, @interval(-10, 10))

elapsed time: 1.017732564 seconds (14999856 bytes allocated)


3-element Array{(Interval{BigFloat},Symbol),1}:
 ([9.999999999999999999999999999999999999999999999999999999999999999999999999999568e-01, 1.000000000000000000000000000000000000000000000000000000000000000000000000000069e+00]₂₅₆,:unique)
 ([1.999999999999999999999999999999999999999999999999999999999999999999999999999914e+00, 2.000000000000000000000000000000000000000000000000000000000000000000000000000207e+00]₂₅₆,:unique)
 ([2.999999999999999999999999999999999999999999999999999999999999999999999999999758e+00, 3.000000000000000000000000000000000000000000000000000000000000000000000000000035e+00]₂₅₆,:unique)

Or we can reuse the results found previously with the lower precision:

In [11]:
setprecision(Interval, 256)
@time roots3 = newton(W₃, roots)

elapsed time: 0.168942833 seconds (2898596 bytes allocated)


3-element Array{(Interval{BigFloat},Symbol),1}:
 ([1e+00, 1e+00]₂₅₆,:unique)                                                                                                                                                              
 ([2e+00, 2e+00]₂₅₆,:unique)                                                                                                                                                              
 ([2.999999999999999999999999999999999999999999999999999999999999999999999999999758e+00, 3.000000000000000000000000000000000000000000000000000000000000000000000000000035e+00]₂₅₆,:unique)

Let's try with a more challenging case. For `Float64`, using the `:wide` rounding is much faster:

In [12]:
setprecision(Interval, Float64)
setrounding(Interval, :narrow)
@time roots = newton(W₁₀, @interval(-20, 20))

elapsed time: 5.750922701 seconds (1619874936 bytes allocated, 27.49% gc time)


10-element Array{(Interval{Float64},Symbol),1}:
 ([0.999999999999997, 1.0000000000000042],:unique) 
 ([1.9999999999997027, 2.0000000000002287],:unique)
 ([2.9999999999975757, 3.000000000003865],:unique) 
 ([3.9999999999716045, 4.000000000029477],:unique) 
 ([4.99999999982392, 5.0000000001432605],:unique)  
 ([5.999999999690147, 6.000000000378698],:unique)  
 ([6.999999999507423, 7.000000000780629],:unique)  
 ([7.999999999455639, 8.00000000084168],:unique)   
 ([8.999999999350004, 9.000000000346732],:unique)  
 ([9.999999999923057, 10.000000000090695],:unique) 

In [13]:
setprecision(Interval, Float64)
setrounding(Interval, :wide)
@time roots = newton(W₁₀, @interval(-20, 20))

elapsed time: 0.602897027 seconds (136606384 bytes allocated, 27.03% gc time)


10-element Array{(Interval{Float64},Symbol),1}:
 ([0.9999999999999781, 1.000000000000022],:unique) 
 ([1.9999999999986873, 2.0000000000015774],:unique)
 ([2.9999999999769145, 3.00000000002351],:unique)  
 ([3.9999999998149893, 4.000000000198532],:unique) 
 ([4.9999999993051, 5.000000000881577],:unique)    
 ([5.999999997587125, 6.000000002318674],:unique)  
 ([6.999999995966628, 7.000000003814153],:unique)  
 ([7.999999996382491, 8.000000004036211],:unique)  
 ([8.999999998178597, 9.000000001849779],:unique)  
 ([9.999999999564606, 10.000000000468848],:unique) 

Using directly higher precision is rather costly:

In [15]:
setprecision(Interval, 256)
@time roots2 = newton(W₁₀, @interval(-20, 20))

elapsed time: 27.047184688 seconds (2373800544 bytes allocated, 49.94% gc time)


10-element Array{(Interval{BigFloat},Symbol),1}:
 ([9.999999999999999999999999999999999999999999999999999999999999999999999999982555e-01, 1.000000000000000000000000000000000000000000000000000000000000000000000000001986e+00]₂₅₆,:unique)
 ([1.999999999999999999999999999999999999999999999999999999999999999999999999897851e+00, 2.000000000000000000000000000000000000000000000000000000000000000000000000132513e+00]₂₅₆,:unique)
 ([2.999999999999999999999999999999999999999999999999999999999999999999999998161947e+00, 3.000000000000000000000000000000000000000000000000000000000000000000000001994126e+00]₂₅₆,:unique)
 ([3.999999999999999999999999999999999999999999999999999999999999999999999986277439e+00, 4.00000000000000000000000000000000000000000000000000000000000000000000001396503e+00]₂₅₆,:unique) 
 ([4.999999999999999999999999999999999999999999999999999999999999999999999933772609e+00, 5.000000000000000000000000000000000000000000000000000000000000000000000064709913e+00]₂₅₆,:unique)
 ([5.99999999999

Whereas reusing the previously-found roots is, of course, much faster:

In [16]:
setprecision(Interval, 256)
@time roots3 = newton(W₁₀, roots)

elapsed time: 0.02859459 seconds (6755144 bytes allocated)


10-element Array{(Interval{BigFloat},Symbol),1}:
 ([9.999999999999999999999999999999999999999999999999999999999999999999999999981691e-01, 1.00000000000000000000000000000000000000000000000000000000000000000000000000171e+00]₂₅₆,:unique) 
 ([1.999999999999999999999999999999999999999999999999999999999999999999999999890355e+00, 2.000000000000000000000000000000000000000000000000000000000000000000000000109645e+00]₂₅₆,:unique)
 ([2.999999999999999999999999999999999999999999999999999999999999999999999998081009e+00, 3.000000000000000000000000000000000000000000000000000000000000000000000001815703e+00]₂₅₆,:unique)
 ([3.999999999999999999999999999999999999999999999999999999999999999999999986651765e+00, 4.000000000000000000000000000000000000000000000000000000000000000000000014338872e+00]₂₅₆,:unique)
 ([4.999999999999999999999999999999999999999999999999999999999999999999999938941667e+00, 5.000000000000000000000000000000000000000000000000000000000000000000000062004858e+00]₂₅₆,:unique)
 ([5.99999999999

However, for very difficult polynomials, even with `Float64` the method starts to suffer. We use the `clean_roots` function to remove identical roots, but even then, there are, for some reason, several almost-repeated roots:

In [17]:
setprecision(Interval, Float64)
setrounding(Interval, :wide)
@time roots = newton(W₁₂, @interval(-20, 20))
roots = ValidatedNumerics.clean_roots(roots)

elapsed time: 2.890292212 seconds (943686744 bytes allocated, 30.37% gc time)


60-element Array{(Interval{Float64},Symbol),1}:
 ([0.9999999999999738, 1.000000000000032],:unique)  
 ([1.9999999999968605, 2.000000000003299],:unique)  
 ([2.999999999937011, 3.0000000000693854],:unique)  
 ([3.9999999992490975, 4.00000000077714],:unique)   
 ([4.999999993998892, 5.0000000058092455],:unique)  
 ([5.999999972275639, 6.000000028158782],:unique)   
 ([6.999999857327201, 7.000000098943499],:unknown)  
 ([6.9999998573272, 7.0000000989435],:unknown)      
 ([6.999999857327201, 7.000000098943499],:unknown)  
 ([6.999999857327202, 7.000000098943498],:unknown)  
 ([7.999999725661288, 8.000000258515469],:unknown)  
 ([7.999999725661287, 8.00000025851547],:unknown)   
 ([7.999999725661288, 8.000000258515469],:unknown)  
 ⋮                                                  
 ([9.999999645289092, 10.000000170076977],:unknown) 
 ([9.999999645289094, 10.000000170076975],:unknown) 
 ([9.999999645289092, 10.000000170076977],:unknown) 
 ([9.999999645289105, 10.000000170076964],:unknown)

We see that the method is unable to resolve several of the roots, and for some reason gives lots of similar, repeated intervals. We can increase the precision as follows, but even using the `clean_roots` function, we still have many repeated roots:

In [20]:
setprecision(Interval, 256)
@time roots2 = newton(W₁₂, roots)
roots2 = ValidatedNumerics.clean_roots(roots2)

elapsed time: 2.106868243 seconds (161580440 bytes allocated, 59.87% gc time)


50-element Array{(Interval{BigFloat},Symbol),1}:
 ([9.999999999999999999999999999999999999999999999999999999999999999999999999975473e-01, 1.000000000000000000000000000000000000000000000000000000000000000000000000002366e+00]₂₅₆,:unique)
 ([1.999999999999999999999999999999999999999999999999999999999999999999999999764993e+00, 2.000000000000000000000000000000000000000000000000000000000000000000000000245854e+00]₂₅₆,:unique)
 ([2.999999999999999999999999999999999999999999999999999999999999999999999994929464e+00, 3.000000000000000000000000000000000000000000000000000000000000000000000005324129e+00]₂₅₆,:unique)
 ([3.999999999999999999999999999999999999999999999999999999999999999999999937344597e+00, 4.000000000000000000000000000000000000000000000000000000000000000000000058873728e+00]₂₅₆,:unique)
 ([4.99999999999999999999999999999999999999999999999999999999999999999999957930183e+00, 5.000000000000000000000000000000000000000000000000000000000000000000000455049532e+00]₂₅₆,:unique) 
 ([5.99999999999